# SB125 Fund Split Analysis

## Question:
- How did RTAs split SB125 funds between operations and capital?

## Methodology:
- upload all avilable `SB125 fund request template` files to gcs
- examine all files for consistencies:
    - come with cleaning plan for inconsistent examples (files withot capital/operating columns)
- concat all rows across all files


## Notes:
- some RTPAs did not submit a `SB125 fund request template.xlsx` file, but instead included an quivilent file their allocation package

In [1]:
import pandas as pd
import os

GCS_PATH = "gs://calitp-analytics-data/data-analyses/sb125/fund_split/"

In [8]:
file_list = [
    "sierra_fund_request.xlsx",
    "slocog_fund_request.xlsx",
    "tehema_fund_request.xlsx",
    "tuolumne_fund_request.xlsx",
    "ventura_fund_request.xlsx",
    "alpine_fund_request.xlsx",
    "amador_fund_request.xlsx",
    "butte_fund_request.xlsx",
    "calaveras_fund_request.xlsx",
    "del_norte_fund_request.xlsx",
    "el_dorado_fund_request.xlsx",
    "humboldt_fund_request.xlsx",
    "kern_fund_request.xlsx",
    "kings_fund_request.xlsx",
    "la_metro_fund_request.xlsx",
    "lake_fund_request.xlsx",
    "lassen_fund_request.xlsx",
    "madera_fund_request.xlsx",
    "mariposa_fund_request.xlsx",
    "mendocino_fund_request.xlsx",
    "merced_fund_request.xlsx",
    "mtc_fund_request.xlsx",
    "nevada_fund_request.xlsx",
    "orange_fund_request.xlsx",
    "placer_fund_request.xlsx",
    "plumas_fund_request.xlsx",
    "riverside_fund_request.xlsx",
    "san_benito_fund_request.xlsx",
    "san_diego_mts_fund_request.xlsx",
    "santa_cruz_fund_request.xlsx",
    "shasta_fund_request.xlsx",
]

file_list.sort()

In [10]:
file_list

['alpine_fund_request.xlsx',
 'amador_fund_request.xlsx',
 'butte_fund_request.xlsx',
 'calaveras_fund_request.xlsx',
 'del_norte_fund_request.xlsx',
 'el_dorado_fund_request.xlsx',
 'humboldt_fund_request.xlsx',
 'kern_fund_request.xlsx',
 'kings_fund_request.xlsx',
 'la_metro_fund_request.xlsx',
 'lake_fund_request.xlsx',
 'lassen_fund_request.xlsx',
 'madera_fund_request.xlsx',
 'mariposa_fund_request.xlsx',
 'mendocino_fund_request.xlsx',
 'merced_fund_request.xlsx',
 'mtc_fund_request.xlsx',
 'nevada_fund_request.xlsx',
 'orange_fund_request.xlsx',
 'placer_fund_request.xlsx',
 'plumas_fund_request.xlsx',
 'riverside_fund_request.xlsx',
 'san_benito_fund_request.xlsx',
 'san_diego_mts_fund_request.xlsx',
 'santa_cruz_fund_request.xlsx',
 'shasta_fund_request.xlsx',
 'sierra_fund_request.xlsx',
 'slocog_fund_request.xlsx',
 'tehema_fund_request.xlsx',
 'tuolumne_fund_request.xlsx',
 'ventura_fund_request.xlsx']

In [ ]:
# loop to extract basename of files
file_name = []

for file in file_list:
    name_extract = os.path.splitext(os.path.basename(file))[0]
    file_name.append(name_extract)

In [ ]:
file_name

In [18]:
def clean_fund_request(file:str) -> pd.DataFrame:
    col_names = [
    "RTPA",
    "Implementing Agenc-y/-ies",
    "Project",
    "Fund Source",
    "capital_FY23-24",
    "capital_FY24-25",
    "capital_FY25-26",
    "capital_FY26-27",
    "operating_FY23-24",
    "operating_FY24-25",
    "operating_FY25-26",
    "operating_FY26-27",
    "total",
]
    df = pd.read_excel(f"{GCS_PATH}{file}", header=2, nrows=40, names=col_names).drop(columns="total")
    df = df.dropna(how= "all")
    return df


In [26]:
def read_in(file:str) -> pd.DataFrame:
    df = pd.read_excel(f"{GCS_PATH}{file}", nrows=40)
    df = df.dropna(how= "all")
    
    if len(df.columns) == 13:
        print(f"{file} can use clean_fund_request func.")
    else: 
        print(f"needs manual check, {file}")
       

In [36]:
def fund_request_checker(file_list:list) -> pd.DataFrame:

    for file in file_list:
    
        df = pd.read_excel(f"{GCS_PATH}{file}", nrows=40)
        df = df.dropna(how= "all")
    
        if len(df.columns) == 13:
            print(f"{file} can use clean_fund_request func.")
        else: 
            print(f"needs manual check, {file}")
       

In [51]:
def fund_request_checker_v2(file_list:list):
    gtg_files = []
    manual_review = []
    for file in file_list:
    
        df = pd.read_excel(f"{GCS_PATH}{file}", nrows=40)
        df = df.dropna(how= "all")
    
        if len(df.columns) == 13:
            gtg_files.append(f"{file}")
        else: 
            manual_review.append(f"{file}")
    return display(
        "good to go files",
        list(gtg_files), 
        "needs manual check", 
        list(manual_review)
    )

In [52]:
def fund_request_checker_v3(file_list:list) -> tuple:
    gtg_files = []
    manual_review = []
    for file in file_list:
    
        df = pd.read_excel(f"{GCS_PATH}{file}", nrows=40)
        df = df.dropna(how= "all")
    
        if len(df.columns) == 13:
            gtg_files.append(f"{file}")
        else: 
            manual_review.append(f"{file}")
    return gtg_files, manual_review

In [13]:
alpine = data("alpine_fund_request.xlsx")

In [22]:
len(alpine.columns)

12

In [14]:
amador = data("amador_fund_request.xlsx")

In [15]:
display(alpine.dtypes, 
        alpine,
        amador.dtypes,
    amador)

RTPA                          object
Implementing Agenc-y/-ies     object
Project                       object
Fund Source                   object
capital_FY23-24              float64
capital_FY24-25              float64
capital_FY25-26              float64
capital_FY26-27              float64
operating_FY23-24            float64
operating_FY24-25            float64
operating_FY25-26            float64
operating_FY26-27            float64
dtype: object

,RTPA,Implementing Agenc-y/-ies,Project,Fund Source,capital_FY23-24,capital_FY24-25,capital_FY25-26,capital_FY26-27,operating_FY23-24,operating_FY24-25,operating_FY25-26,operating_FY26-27
0,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,TIRCP,360641.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,TIRCP,NaN,360794.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,ZETCP (GGRF),3616.684,3781.0,3781.0,3781.0,NaN,NaN,NaN,NaN
3,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,ZETCP (PTA),3123.316,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,Grand Total,NaN,NaN,NaN,367381.000,364575.0,3781.0,3781.0,0.0,0.0,0.0,0.0


RTPA                          object
Implementing Agenc-y/-ies     object
Project                      float64
Fund Source                   object
capital_FY23-24              float64
capital_FY24-25              float64
capital_FY25-26              float64
capital_FY26-27              float64
operating_FY23-24            float64
operating_FY24-25            float64
operating_FY25-26            float64
operating_FY26-27            float64
dtype: object

,RTPA,Implementing Agenc-y/-ies,Project,Fund Source,capital_FY23-24,capital_FY24-25,capital_FY25-26,capital_FY26-27,operating_FY23-24,operating_FY24-25,operating_FY25-26,operating_FY26-27
2,Amador County Transportation Commission,Amador County Transportation Commission,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Amador Transit,NaN,TIRCP,100000.0,75000.0,NaN,NaN,NaN,NaN,NaN,NaN
20,Grand Total,NaN,NaN,NaN,100000.0,75000.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
fund_request_checker(file_list)

In [ ]:
fund_request_checker_v2(file_list)

In [53]:
good_list, review_list = fund_request_checker_v3(file_list)

In [55]:
display(
    len(good_list),
    len(review_list)
)

24

7